In [55]:
import paramiko
from randcrack import RandCrack
import time

def hex_to_bits(hex_str):
    return bin(int(hex_str, 16))[2:].zfill(64)

def main():
    # Setup SSH
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect('pe111.comp.nus.edu.sg', username='e1121756', password='crabllama10;')
    
    channel = ssh.invoke_shell()
    time.sleep(1)
    channel.send('nc cs2107-ctfd-i.comp.nus.edu.sg 8001\n')
    time.sleep(1)
    
    # Initialize cracker
    rc = RandCrack()
    
    # Clear initial prompt
    while not channel.recv_ready():
        time.sleep(0.1)
    print(channel.recv(1024).decode())
    
    # Step 1: Collect exactly 312 samples (leaving room for verification)
    print("Collecting samples...")
    samples = []
    
    for i in range(312):
        channel.send('0\n')
        time.sleep(0.1)
        
        response = ''
        while 'Your guess: ' not in response:
            if channel.recv_ready():
                response += channel.recv(1024).decode()
        
        hex_value = response.split('was ')[1].split('.')[0]
        samples.append(hex_value)
        print(f"Sample {i+1}: {hex_value}")
        
        # Convert to bits and feed to predictor
        byte = bytes.fromhex(hex_value)
        
        #bits = hex_to_bits(hex_value)
        first_4 = byte[:4]
        second_4 = byte[4:]
        rc.submit(int.from_bytes(first_4, 'little'))
        rc.submit(int.from_bytes(second_4, 'little'))
    
    # # Step 2: Get verification sample and use it for last prediction setup
    # print("\nGetting verification sample...")
    # channel.send('0\n')
    # time.sleep(0.1)
    # response = ''
    # while 'Your guess: ' not in response:
    #     if channel.recv_ready():
    #         response += channel.recv(1024).decode()
    # verify_value = response.split('was ')[1].split('.')[0]
    # print(f"Verification value: {verify_value}")
    
    # # Feed verification value to predictor
    # # bits = hex_to_bits(verify_value)
    # # first_32 = int(bits[:32], 2)
    # # second_32 = int(bits[32:], 2)
    # # rc.submit(first_32)
    # # rc.submit(second_32)
    
    # # Step 3: Make prediction
    prediction = rc.predict_getrandbits(64).to_bytes(8, 'little').hex()
    #hex_prediction = hex(prediction)[2:].zfill(16)
    print(f"Predicting: {prediction}")
    
    # Send prediction
    channel.send(f"{prediction}\n")
    time.sleep(1)
    
    # Get result
    print("\nFinal response:")
    while channel.recv_ready():
        print(channel.recv(1024).decode())
    
    channel.close()
    ssh.close()

if __name__ == "__main__":
    main()

Welcome to Ubuntu 20.04.6 LTS (GNU/Linux 5.4.0-205-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/pro

 System information as of Wed 12 Feb 2025 03:23:40 PM +08

  System load:  1.34               Processes:             388
  Usage of /:   19.2% of 46.54GB   Users logged in:       11
  Memory usage: 9%                 IPv4 address for eno1: 192.168.48.113
  Swap usage:   0%

 * Strictly confined Kubernetes makes edge and IoT secure. Learn how MicroK8s
   just raised the bar for easy, resilient and secure K8s cluster deployment.

   https://ubuntu.com/engage/secure-kubernetes-at-the-edge

Expanded Security Maintenance for Applications is not enabled.

0 updates can be applied immediately.

29 additional security updates can be applied with ESM Apps.
Learn more about enabling ESM Apps service at https://ubuntu.com/esm

New release '22.04.5 LTS' available.
Run 'do-release-upgrade' to up
S

In [54]:
import random, time
from randcrack import RandCrack

random.seed(time.time())

rc = RandCrack()

for i in range(624):
	rc.submit(int.from_bytes(random.randbytes(4), 'little'))
	#rc.submit(int(random.randbytes(4).hex(), 16))
	# Could be filled with random.randint(0,4294967294) or random.randrange(0,4294967294)

print("Random result: {}\nCracker result: {}"
	.format(random.randbytes(8), rc.predict_getrandbits(64).to_bytes(8, 'little')))

Random result: b'\xb1\xf9\x84\x95\xe2T\xee\x0c'
Cracker result: b'\xb1\xf9\x84\x95\xe2T\xee\x0c'


In [37]:
test = random.randbytes(4)

In [47]:
len(bytes.fromhex(test.hex()))

4

In [45]:
int.from_bytes(test, 'little')

4001733648